# **INTRODUCTIONS**

---

The source code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb",
        "src/statics/populations/*",
        "src/statics/calendar.ipynb"
    ]
}
```

# **TABLES**

---


### STAGING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE DEST SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSSTAGE_RIVM_SEWER_MEASUREMENTS_2021]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSSTAGE_RIVM_SEWER_MEASUREMENTS_2021]
	START WITH 1
	INCREMENT BY 1;
GO

-- 2) CREATE DEST TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE OBJECT_ID = OBJECT_ID(N'[VWSSTAGE].[RIVM_SEWER_MEASUREMENTS]'))
CREATE TABLE [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS] (
	[ID] [INT] PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTAGE_RIVM_SEWER_MEASUREMENTS_2021]),
	[VERSION] [VARCHAR](100) NULL,
	[DATE_MEASUREMENT] [VARCHAR](100) NULL,
	[RWZI_AWZI_CODE] [VARCHAR](100) NULL,
	[RWZI_AWZI_NAME] [VARCHAR](100) NULL,
	[RNA_PER_ML] [VARCHAR](100) NULL,
	[RNA_FLOW_PER_100000] [VARCHAR](100) NULL,
	[DATE_OF_REPORT] [VARCHAR](100) NULL,
	[DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE()
);
GO

### INTERMEDIATES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE DEST SEQUENCE(S).....
IF NOT EXISTS (SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSINTER_RIVM_SEWER_MEASUREMENTS_2021]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSINTER_RIVM_SEWER_MEASUREMENTS_2021]
    START WITH 1
    INCREMENT BY 1;
GO

-- 2) CREATE DEST TABLE(S).....
IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[RIVM_SEWER_MEASUREMENTS]'))
CREATE TABLE [VWSINTER].[RIVM_SEWER_MEASUREMENTS](
	[ID] [INT] PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSINTER_RIVM_SEWER_MEASUREMENTS_2021]),
	[DATE_MEASUREMENT] [DATETIME],
	[VERSION] [VARCHAR](100) NULL,
	[RWZI_AWZI_CODE] [BIGINT],
	[RWZI_AWZI_NAME] [VARCHAR](100) NULL,
	[RNA_PER_ML] [BIGINT],
	[RNA_FLOW_PER_100000] [BIGINT],
	[DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE()
);
GO

### DESTINATIONS

> SEWER PROLONGATED

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE DEST SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSDEST_SEWER_PROLONGATED]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSDEST_SEWER_PROLONGATED]
  START WITH 1
  INCREMENT BY 1;
GO

-- 2) CREATE DEST TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[SEWER_PROLONGATED]'))
CREATE TABLE [VWSDEST].[SEWER_PROLONGATED](
  [ID] [INT] PRIMARY KEY NONCLUSTERED NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSDEST_SEWER_PROLONGATED]),
  [DATE_MEASUREMENT] [DATETIME] NULL,               
  [REGIO_CODE] [VARCHAR](10) NULL,          
  [WEEK] [INT] NULL,          
  [WEEK_UNIX] [BIGINT] NULL,      
  [AVERAGE_RNA_FLOW_PER_100000] [DECIMAL](16,3) NULL,          
  [REGION_COVERED_INHABITANTS] [INT] NULL,                                                              
  [DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

-- 3) CREATE DEST INDEX(ES).....
IF NOT EXISTS(SELECT * FROM SYS.INDEXES WHERE [NAME] = 'CI_DLI_VWSDEST_SEWER_PROLONGATED')
CREATE CLUSTERED INDEX [CI_DLI_VWSDEST_SEWER_PROLONGATED] ON [VWSDEST].[SEWER_PROLONGATED] (
  [DATE_LAST_INSERTED]
);
GO

IF NOT EXISTS(SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCI_VWSDEST_CIMS_VACCINATED_AGE_GROUP')
CREATE NONCLUSTERED INDEX [NCI_VWSDEST_CIMS_VACCINATED_AGE_GROUP] 
ON [VWSDEST].[SEWER_PROLONGATED] ([DATE_LAST_INSERTED],[DATE_MEASUREMENT],[REGIO_CODE])  
    INCLUDE (
    [AVERAGE_RNA_FLOW_PER_100000],
    [REGION_COVERED_INHABITANTS]
    );
GO

> SEWER BASE

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE DEST SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSDEST_SEWER_BASE_2021]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSDEST_SEWER_BASE_2021]
  START WITH 1
  INCREMENT BY 1;
GO

-- 2) CREATE DEST TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[SEWER_BASE]'))
CREATE TABLE [VWSDEST].[SEWER_BASE](
	[ID] [INT] PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSDEST_SEWER_BASE_2021]),
	[WEEK] [INT] NULL,
	[WEEK_UNIX] [BIGINT] NULL,
  [DATE_MEASUREMENT] [DATETIME] NULL,
  [RWZI_AWZI_CODE] [INT] NULL,
  [RWZI_AWZI_NAME] [VARCHAR](100) NULL,
  [GM_CODE] [VARCHAR](100) NULL,
  [RNA_PER_ML] [BIGINT],
  [RNA_FLOW_PER_100000] [BIGINT] NULL,
  [COUNT_DAILY_MEASUREMENT] [INT] NULL,
  [DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

# **STORED PROCEDURES**

---


### INTERMEDIATE &rarr; DESTINATION MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_SEWER_PROLONGATED]
AS
BEGIN
WITH BASE_CTE AS (

    SELECT 
         WEEK
        ,DATE_MEASUREMENT
        ,RWZI_AWZI_CODE 
        ,RWZI_AWZI_NAME
        ,RNA_FLOW_PER_100000 
        ,RNA_FLOW_PER_100000 / 1.0E+11 AS RNA_FLOW_PER_100000_PER_RWZI
    FROM VWSDEST.SEWER_BASE
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.SEWER_BASE)
        -- KEEP ONLY THE FIRST MEASUREMENT (SOME DUPLICATE MEASUREMENTS PER DAY PER RWZI ARE PRESENT)
        AND COUNT_DAILY_MEASUREMENT=1
)
,
RWZI_CTE AS (
    -- ALLOCATION TABLE WITH DATA ON GM AND VR LEVEL
    SELECT 
        [RWZI_CODE],
        [RWZI_NAAM],
        [STARTDATUM],
        [EINDDATUM],
        [INWONERS],
        [REGIO_TYPE],
        [REGIO_CODE],
        [REGIO_NAAM],
        [AANDEEL],
        [INWONERS] * ([AANDEEL]/100) AS [COVERED_INHABITANTS]
    FROM VWSSTATIC.CBS_POPULATION_RWZI
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_RWZI)
        AND REGIO_TYPE = 'VR' -- GM WILL NOT BE USING THIS METHOD.
    
    UNION ALL

        SELECT DISTINCT
        [RWZI_CODE],
        [RWZI_NAAM],
        [STARTDATUM],
        [EINDDATUM],
        [INWONERS],
        'NL'            AS [REGIO_TYPE],
        'NL'            AS [REGIO_CODE],
        'Netherlands'   AS [REGIO_NAAM],
        100.00          AS [AANDEEL],
        [INWONERS]      AS [COVERED_INHABITANTS]
    FROM VWSSTATIC.CBS_POPULATION_RWZI
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_RWZI)
        AND REGIO_TYPE = 'VR' -- CHOOSE EITHER VR OR GM FOR RARE CASES WHERE INHABITANTS DO NOT ALIGN SUCH AS RWZI 9041

),
GRID_CTE AS (
    -- USE CALENDAR TABLE TO CREATE A RECORD FOR EACH POSSIBLE COMBINATION OF DATE, RWZI AND REGION
SELECT  
         T0.DATE_MEASUREMENT
        ,T1.RWZI_CODE
        ,T1.[REGIO_CODE]
        ,T1.[REGIO_NAAM]
    FROM 
        (
            SELECT CAST([DAY] AS DATETIME) AS  [DATE_MEASUREMENT] FROM [VWSSTATIC].[CALENDAR]
                WHERE   [DAY]>= (SELECT MIN(DATE_MEASUREMENT) FROM BASE_CTE)
                AND [DAY]<= (SELECT MAX(DATE_MEASUREMENT) FROM BASE_CTE)
        ) AS T0
    CROSS JOIN 
        (SELECT DISTINCT 
             [RWZI_CODE]
            ,[REGIO_CODE]
            ,[REGIO_NAAM]
        FROM RWZI_CTE) AS T1
    --ADD ADDITIONAL (DE-DUPLICATED) INFO TO RWZI AND REGIO DIMENSIONS
),
DAILY_CTE AS (
    SELECT 
         T0.WEEK
        ,T0.DATE_MEASUREMENT
        ,T0.RWZI_AWZI_CODE
        ,T0.RWZI_AWZI_NAME
        -- ,T0.RNA_FLOW_PER_100000 
        ,T0.RNA_FLOW_PER_100000_PER_RWZI
        ,T1.STARTDATUM
        ,T1.EINDDATUM
        ,T1.REGIO_CODE
        ,T1.AANDEEL
        ,T1.INWONERS
        ,T1.COVERED_INHABITANTS
    FROM BASE_CTE AS T0
    LEFT JOIN RWZI_CTE AS T1
        ON  T0.RWZI_AWZI_CODE = T1.RWZI_CODE
        AND T1.STARTDATUM <= T0.DATE_MEASUREMENT AND T1.EINDDATUM > T0.DATE_MEASUREMENT
),
SPARSE_CTE AS
(
    SELECT 
         T0.DATE_MEASUREMENT
        ,T0.RWZI_CODE
        ,T0.[REGIO_CODE]
        ,T0.[REGIO_NAAM]
        ,'|' AS [|]
        ,T1.WEEK
        ,T1.RWZI_AWZI_NAME
        ,T1.RNA_FLOW_PER_100000_PER_RWZI AS RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL
        ,T1.AANDEEL
        ,T1.INWONERS
        ,T1.COVERED_INHABITANTS
        --DETERMINE THE LAST DATE THAT CONTAINED VALUES FOR THIS RWZI AND REGION
        ,MAX(CASE WHEN T1.REGIO_CODE IS NOT NULL THEN T0.[DATE_MEASUREMENT] END)
            OVER (PARTITION BY T0.RWZI_CODE, T0.REGIO_CODE ORDER BY T0.[DATE_MEASUREMENT] ROWS 8 PRECEDING) AS PROLONGATE_DATE
    FROM GRID_CTE AS T0
    LEFT JOIN DAILY_CTE AS T1 ON T0.DATE_MEASUREMENT = T1.DATE_MEASUREMENT AND T0.RWZI_CODE = T1.RWZI_AWZI_CODE AND T0.REGIO_CODE = T1.REGIO_CODE
),
DENSE_CTE AS (
SELECT 
     *
     --ADD THE TOTAL NUMBER OF INHABITANTS IN THE REGION BASED ON THE PROLONGATED INHABITANT NUMBERS
    ,SUM(COVERED_INHABITANTS)                   OVER (PARTITION BY            REGIO_CODE, DATE_MEASUREMENT) AS REGION_COVERED_INHABITANTS
    FROM (
        --PROLONGATE THE VALUES: RETRIEVE THE ONLY VALUE DETERMINED FOR PROLONGATE DATE, RWZI AND REGION COMBINATION
        SELECT 
             DATE_MEASUREMENT
            ,RWZI_CODE
            ,[REGIO_CODE]
            ,[REGIO_NAAM]
            ,[|]
            ,MAX(RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL) OVER (PARTITION BY RWZI_CODE, REGIO_CODE, PROLONGATE_DATE) AS RNA_FLOW_PER_100000_PER_RWZI
            ,MAX(COVERED_INHABITANTS) OVER (PARTITION BY RWZI_CODE, REGIO_CODE, PROLONGATE_DATE) AS COVERED_INHABITANTS
            ,AANDEEL
            ,INWONERS
            -- ,COVERED_INHABITANTS
            ,PROLONGATE_DATE
        FROM SPARSE_CTE
        ) T0   
)
    INSERT INTO VWSDEST.SEWER_PROLONGATED(
         [DATE_MEASUREMENT]                     
        ,[REGIO_CODE]                           
        ,[WEEK]                                 
        ,[WEEK_UNIX]                            
        ,[AVERAGE_RNA_FLOW_PER_100000]          
        ,[REGION_COVERED_INHABITANTS]      
    )
    SELECT 
        [DATE_MEASUREMENT]
        ,[REGIO_CODE]
        ,DATEPART(ISO_WEEK, [DATE_MEASUREMENT]) AS WEEK
        ,[DBO].[CONVERT_DATETIME_TO_UNIX](DATEADD(WEEK, DATEDIFF(WEEK, 0, DATE_MEASUREMENT - 1), 0)) AS WEEK_UNIX
        ,CASE  
            WHEN [REGION_COVERED_INHABITANTS] != 0 THEN SUM( (COVERED_INHABITANTS * RNA_FLOW_PER_100000_PER_RWZI) ) / REGION_COVERED_INHABITANTS
            ELSE  SUM( ([COVERED_INHABITANTS] * RNA_FLOW_PER_100000_PER_RWZI) ) /1 
        END 
        AS [AVERAGE_RNA_FLOW_PER_100000]
        ,[REGION_COVERED_INHABITANTS]
    FROM DENSE_CTE
    GROUP BY DATE_MEASUREMENT, [REGIO_CODE], REGION_COVERED_INHABITANTS;
END;
GO